In [ ]:
# This cell installs Grounding DINO and its requirements.
# Note: You must enable GPU in Colab (Runtime -> Change runtime type -> T4 GPU)

import os

# Install Grounding DINO from source (Official Repo)
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd GroundingDINO/
!pip install -q .

# Install supervision for easy visualization
!pip install -q supervision==0.6.0

# Download Pre-trained Weights
if not os.path.exists("weights"):
    os.makedirs("weights")
!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth -P weights/

print("Installation and Download Complete.")

In [ ]:
import torch
from groundingdino.util.inference import load_model, load_image, predict, annotate
import supervision as sv

# Paths
CONFIG_PATH = "groundingdino/config/GroundingDINO_SwinT_OGC.py"
WEIGHTS_PATH = "weights/groundingdino_swint_ogc.pth"

model = load_model(CONFIG_PATH, WEIGHTS_PATH)
print("Model Loaded Successfully")

In [ ]:
import cv2
import matplotlib.pyplot as plt

def run_demo(image_path, text_prompt, box_threshold=0.35, text_threshold=0.25):
    print(f"Running inference on: {image_path} with prompt: '{text_prompt}'")

    # Load image
    image_source, image = load_image(image_path)

    # Predict
    boxes, logits, phrases = predict(
        model=model,
        image=image,
        caption=text_prompt,
        box_threshold=box_threshold,
        text_threshold=text_threshold
    )

    # Annotate
    annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
    
    # Display using Matplotlib (Colab doesn't support cv2.imshow)
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(f"Prompt: {text_prompt}")
    plt.show()

In [ ]:
# INSTRUCTIONS: 
# 1. Click the folder icon on the left sidebar.
# 2. Drag and drop your 'samples' folder (containing images) there.

# Example 1: Aerial
if os.path.exists("../samples/aerial_demo.jpg"):
    run_demo("../samples/aerial_demo.jpg", "aerial view of objects")
else:
    print("Please upload the 'samples' folder to the Colab runtime!")

# Example 2: Video Game
if os.path.exists("../samples/game_demo.jpg"):
    run_demo("../samples/game_demo.jpg", "character, weapon")

# Example 3: Medical (Showing failure/success)
if os.path.exists("../samples/medical_demo.jpg"):
    run_demo("../samples/medical_demo.jpg", "x-ray, bone")